<a href="https://colab.research.google.com/github/RichardXiao13/Google_Code_In/blob/master/ProgressBar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Progress Bar**

# **Setup**

In [0]:
try:
  %tensorflow_version 2.x
except:
  Exception
import tensorflow as tf

TensorFlow 2.x selected.


# **Create Bar**
We will need to use some imports that will allow us to display a progress bar during training.

In [0]:
from IPython.display import HTML, display
def progress(value, max, metrics=""):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 40%'
        >
        </progress> {value}/{max}  {metrics}
    """.format(value=value, max=max, metrics=metrics))

def metric(metrics):
  return HTML("""
      {metrics}
      """.format(metrics=metrics))

# **Time**
We will import time to keep track of training duration.

In [0]:
import time

# **Callback**
Create the ProgressBar class which will log the metrics during training.

In [0]:
class ProgressBar(tf.keras.callbacks.Callback):
  def __init__(self, batches):
    self.batches = batches
  def on_train_begin(self, logs=None):
    self.epochs = self.params['epochs']
    self.eta = display("Training ETA: Unknown", display_id=True)
    self.master = display(progress(0, self.epochs), display_id=True)
  
  def on_epoch_begin(self, epoch, logs=None):
    if epoch != 0:
      self.timeBar.update("ETA: 0.0 s")
    self.timeStart = time.time()
    self.master.update(progress(epoch+1, self.epochs))
    self.bar = display(progress(0, self.batches), display_id=True)
    self.timeBar = display("ETA: Unknown", display_id=True)
  
  def on_epoch_end(self, epoch, logs=None):
    self.timeEnd = time.time()
    self.trainTime = (self.timeEnd - self.timeStart) * (self.epochs - epoch + 1)
    self.eta.update("Training ETA: " + str(self.trainTime) + " s")
    self.loss = round(logs.get("loss"), 5)
    self.acc = round(logs.get("accuracy"), 5)
    self.val_loss = round(logs.get("val_loss"), 5)
    self.val_acc = round(logs.get("val_accuracy") * 100, 5)
    self.bar.update("Epoch " + str(epoch+1) + " Loss: " + str(self.loss) + " Accuracy: " + str(self.acc) + " Validation Loss: " + str(self.val_loss) + " Validation Accuracy: " + str(self.val_acc) + "%")

  def on_batch_begin(self, batch, logs=None):
    self.batchBegin = time.time()

  def on_batch_end(self, batch, logs=None):
    self.batchEnd = time.time()
    self.batchTime = (self.batchEnd - self.batchBegin) * (self.batches - batch + 1)
    self.bar.update(progress(batch+1, self.batches))
    self.timeBar.update("ETA: " + str(self.batchTime) + " s")
  
  def on_train_end(self, logs=None):
    self.eta.update("ETA: 0.0 s")
    self.timeBar.update("ETA: 0.0 s")
    self.finished = display("Training Complete!", display_id=True)


Create an object of the class.

In [0]:
cb = ProgressBar(60)

# **Try it out**
We will load the  fashion mnist dataset and train a model to try our progress bar.

In [0]:
import tensorflow_datasets as tfds

In [0]:
train_data, test_data = tfds.load("fashion_mnist", as_supervised=True, split=["train", "test"])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset fashion_mnist downloaded and prepared to /root/tensorflow_datasets/fashion_mnist/1.0.0. Subsequent calls will reuse this data.


In [0]:
train_data = train_data.batch(1000).prefetch(1)
test_data = test_data.batch(1000)

In [0]:
import tensorflow.keras.layers as layers

In [0]:
model = tf.keras.Sequential([
                             layers.Input(shape=(28, 28, 1)),
                             layers.Convolution2D(16, (3, 3), padding="same", activation="relu"),
                             layers.MaxPooling2D(),
                             layers.Convolution2D(32, (3, 3), padding="same", activation="relu"),
                             layers.Flatten(),
                             layers.Dense(128, activation="relu"),
                             layers.Dense(10, activation="softmax")
])

In [0]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [0]:
model.fit(train_data, validation_data=test_data, epochs=10, verbose=0, callbacks=[cb])

'ETA: 0.0 s'

'Epoch 1 Loss: 12.35681 Accuracy: 0.64865 Validation Loss: 0.72642 Validation Accuracy: 79.66%'

'ETA: 0.0 s'

'Epoch 2 Loss: 0.50509 Accuracy: 0.83908 Validation Loss: 0.44477 Validation Accuracy: 84.9%'

'ETA: 0.0 s'

...

'Epoch 9 Loss: 0.18326 Accuracy: 0.93485 Validation Loss: 0.33789 Validation Accuracy: 88.63%'

'ETA: 0.0 s'

'Epoch 10 Loss: 0.16921 Accuracy: 0.93947 Validation Loss: 0.3419 Validation Accuracy: 88.69%'

'ETA: 0.0 s'

'Training Complete!'

# **Results**
The progress bar works as expected and displays all the important information!